문제 4.
인터파크 티켓 예매 페이지에서 인기 공연 데이터를 수집하고 공연 카테고리별 소비 분석
https://nol.interpark.com/ticket
1) (카테고리) 공연명, 날짜, 장소, 가격 저장
3) SQL로 분석해야 하는 내용
**장르별** 평균 가격과 최저·최고 가격 -> 뮤지컬대비 연극 가격 같이 카테고리별 가격대를 구별할 수 있도록 : 마케팅
0일 이내 시작하는 공연 중 가격 상위 5개 -> 현시점 기준. 가장 최신공연! 
공연 장소별 등록된 공연 수 순위

In [9]:
###인터파크에서 요즘 HOT 콘서트에서 클로링해오기!!!!###
# 1, 싱글페이지 어플리케이션인지는 크롬확장프로그램으로 확인!(원자모양)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

service = Service(ChromeDriverManager().install()) #서비스 실행되게 세팅!

options = Options();
#options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")

#실질적인 크롤링 역할
driver = webdriver.Chrome(service=service, options=options) #웹드라이버에서 각각의 역할을 하게 세팅

url = "https://nol.interpark.com/ticket"
driver.get(url)
time.sleep(2) 

try : 
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause = 1
except Exception as e: #이 실행이 에러가 나면 except문을 실행시켜라!
    print("스크롤 내림 실패 : ", e)

for _ in range(3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(scroll_pause)
    #스크롤했는데, 다음 높이값이 더 추가되었을때 (보험용) 
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:#만약 새로운높이값이 나오면 마지막스크롤에 닿으면 멈춰라
        break
    last_height = new_height  #else문, else 생략. 
        #스크롤 할때마다 높이가 달라지는 페이지에서 쓰이는 문구!
cards = driver.find_elements(By.CSS_SELECTOR,"dl.ranking_info__DR7GO")
for i, card in enumerate(cards,1) :
    titles = card.find_elements(By.CSS_SELECTOR,"dl.ranking_info__DR7GO > dt")
    dates = card.find_elements(By.CSS_SELECTOR,"dl.ranking_info__DR7GO > dd")
    places= card.find_elements(By.CSS_SELECTOR,"dl.ranking_info__DR7GO > dd")
    
print(f"{i}. {titles},{dates},{places}")

data = []

# for title in titles :
#     a = dict(title)
#     for date in dates:
#         a.values(date) -> 비효율적!!
#한번에 돌고 돌때마다 같은 인덱스에서 나온 값을 병렬로 직결시켜줄게! = zip
for t,d,p in zip(titles,dates,places):
    data.append({
        "title":t.text.strip(),
        "date":d.text.strip(),
        "place":p.text.strip()
    }) #딕셔너리 형태로 넣어놔야 행열로 쉽게 넣을 수 있음!

#print(data)

driver.quit()

# 7, SQL로 보내기
#어떠한데이터를 중심적으로 움직이는 지확인하는 판다스의 데이터프레임
df = pd.DataFrame(data)
df.head()
#이렇게 전처리를 마치면 SQL로 보내기 쉬워짐!!!! => 전처리끝!
# => MYSQL로 가서 데이터를 담을 데이터베이스 & 테이블에 담기! 250801 - SQL05

10. [<selenium.webdriver.remote.webelement.WebElement (session="5d0864be107c0e53c07874b324713500", element="f.EE87E8FC508CE991850C74574B41B13A.d.4EB2A5EF596C9B286672F16638D43B57.e.623")>],[<selenium.webdriver.remote.webelement.WebElement (session="5d0864be107c0e53c07874b324713500", element="f.EE87E8FC508CE991850C74574B41B13A.d.4EB2A5EF596C9B286672F16638D43B57.e.624")>, <selenium.webdriver.remote.webelement.WebElement (session="5d0864be107c0e53c07874b324713500", element="f.EE87E8FC508CE991850C74574B41B13A.d.4EB2A5EF596C9B286672F16638D43B57.e.625")>],[<selenium.webdriver.remote.webelement.WebElement (session="5d0864be107c0e53c07874b324713500", element="f.EE87E8FC508CE991850C74574B41B13A.d.4EB2A5EF596C9B286672F16638D43B57.e.624")>, <selenium.webdriver.remote.webelement.WebElement (session="5d0864be107c0e53c07874b324713500", element="f.EE87E8FC508CE991850C74574B41B13A.d.4EB2A5EF596C9B286672F16638D43B57.e.625")>]


,title,date,place
0,,,


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd

service = Service(ChromeDriverManager().install()) #서비스 실행되게 세팅!

options = Options();
#options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")

#실질적인 크롤링 역할
driver = webdriver.Chrome(service=service, options=options )#웹드라이버에서 각각의 역할을 하게 세팅

url = "https://nol.interpark.com/ticket"
driver.get(url)
time.sleep(2) 

try : 
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause = 1
except Exception as e: #이 실행이 에러가 나면 except문을 실행시켜라!
    print("스크롤 내림 실패 : ", e)

for _ in range(3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(scroll_pause)
    #스크롤했는데, 다음 높이값이 더 추가되었을때 (보험용) 
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:#만약 새로운높이값이 나오면 마지막스크롤에 닿으면 멈춰라
        break
    last_height = new_height  #else문, else 생략. 
        #스크롤 할때마다 높이가 달라지는 페이지에서 쓰이는 문구!
titles = card.find_elements(By.CSS_SELECTOR, "dl.ranking_info__DR7GO > dt")
dates = card.find_elements(By.CSS_SELECTOR, "dl.ranking_info__DR7GO > dd")

for t, d in zip(titles, dates):
    print(f"{i}. {t.text.strip()} , {d.text.strip()}")

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x11bba83+63395]
	GetHandleVerifier [0x0x11bbac4+63460]
	(No symbol) [0x0x1002113]
	(No symbol) [0x0xff1b20]
	(No symbol) [0x0x100f922]
	(No symbol) [0x0x1075c9c]
	(No symbol) [0x0x1090099]
	(No symbol) [0x0x106f1a6]
	(No symbol) [0x0x103e7b2]
	(No symbol) [0x0x103f654]
	GetHandleVerifier [0x0x1438883+2672035]
	GetHandleVerifier [0x0x1433cba+2652634]
	GetHandleVerifier [0x0x11e2bca+223466]
	GetHandleVerifier [0x0x11d2cb8+158168]
	GetHandleVerifier [0x0x11d978d+185517]
	GetHandleVerifier [0x0x11c3b78+96408]
	GetHandleVerifier [0x0x11c3d02+96802]
	GetHandleVerifier [0x0x11ae90a+9770]
	BaseThreadInitThunk [0x0x755c5d4

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchAttributeException
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent = Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

categorys = ["001001","001005","001004"]

for category in categorys:
    url = f"https://display.wconcept.co.kr/category/women/{query}"
    driver.get(url)
    time.sleep(2)
    print(f"\n\n-----------{category} 카테고리 ------")

    idx = 0

    while idx < 31:
        try:
            try : 
                last_height = driver.execute_script("return document.body.scrollHeight")
                scroll_pause = 1
            except Exception as e: 
                print("스크롤 내림 실패 : ", e)
            
            for _ in range(3):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
                time.sleep(scroll_pause)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            cards = driver.find_elements(By.CSS_SELECTOR,"span.sc-jpuob9-6.ejEjBO.area-info")
            for i, card in enumerate(cards, 1):
                try:
                    title = card.find_element(By.CSS_SELECTOR, "span.sc-tdjdw3-0.frpBAf.text.title")
                except:
                    title = ""                
                try:
                    score = card.find_element(By.CSS_SELECTOR, "em.score")
                except:
                    score = ""
                try:
                    review = card.find_element(By.CSS_SELECTOR, "span.sc-tdjdw3-0.frpBAf.text.cnt")
                except:
                    review = ""                
           
                print(f"{i}. {titles.text.strip()},{score.text.strip()},{review.text.strip()}")
            
            card = []
    
            time.sleep(2)
            idx += 1
        except StaleElementReferenceException:
            print(f"{idx+1}번째 요소 실패. 재시도")
            time.sleep(1)
            continue
time.sleep(1)
driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import mysql.connector

service = Service(ChromeDriverManager().install())

options=Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--suer-agent")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

categorys = ["001005","001002","001008"]

for category in categorys:
    url = f"https://display.wconcept.co.kr/category/women/{category}"
    driver.get(url)
    time.sleep(2)

    print(f"------ {category} 크롤링-------")

    idx=0
    while idx <30:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR,"button.area-img")
            if idx >= len(buttons):
                break
            driver.execute_script("arguments[0].click();",buttons[idx])
            time.sleep(3)

            print(f"\n---- {category} {idx+1}번째 의류")
            try:
                title = driver.find_element(By.CSS_SELECTOR,"h2.brand").text.strip()
            except NoSuchElementException:
                title = "NOTHING"
                
            try:
                rate = driver.find_element(By.CSS_SELECTOR,"p.grade_num").text.strip()
            except NoSuchElementException:
                title = "0"
                
            try:
                review_num = driver.find_element(By.CSS_SELECTOR,"#reviewCnt1").text.strip()
            except NoSuchElementException:
                review_num = "0"

            try:
                driver.execute_script("window.scrollBy(0,300);")
                time.sleep(1)
                review_button = driver.find_element(By.CSS_SELECTOR,"#detail > li:nth-child(2)")
                review_button.click()
                time.sleep(2)
                review_elements = driver.find_elements(By.CSS_SELECTOR,"p.pdt_review_text")
                reviews = [r.text.strip() for r in review_elements[:2]]

                if not reviews:
                    reviews = ["NO reviews"]
            except NoSuchElementException: 
                reviews = ["NO reviews"]
            print("상품 : ", title)
            print("평점 : ", title)
            print("리뷰수 : ", title)

            for i, review in reviews:
                print(f"리뷰{i}: ", review)

            driver.back()
            time.sleep(1)

            driver.back()
            time.sleep(1)
            
            driver.execute_script("window.scrollBy(0,200);")
            time.sleep(1)

            idx+= 1
        except StaleElementReferenceException:
            print(f"{idx+1}번째 요소 stale 참조 발생, 재시도")

            time.sleep(1)
            continue

driver.quit()